In [2]:
%logstop
%logstart -rtq ~/.logs/pw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [3]:
from static_grader import grader

# PW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to use basic Python data structures, define functions, and control program flow. We will be using these concepts to perform some fundamental data wrangling tasks such as joining data sets together, splitting data into groups, and aggregating data into summary statistics.
**Please do not use `pandas` or `numpy` to answer these questions.**

We will be working with medical data from the British NHS on prescription drugs. Since this is real data, it contains many ambiguities that we will need to confront in our analysis. This is commonplace in data science, and is one of the lessons you will learn in this miniproject.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [3]:
%%bash
mkdir pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz -nc -P ./pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/practices.json.gz -nc -P ./pw-data

mkdir: cannot create directory ‘pw-data’: File exists
File ‘./pw-data/201701scripts_sample.json.gz’ already there; not retrieving.

File ‘./pw-data/practices.json.gz’ already there; not retrieving.



## Loading the data

The first step of the project is to read in the data. We will discuss reading and writing various kinds of files later in the course, but the code below should get you started.

In [ ]:
import gzip
import simplejson as json

In [ ]:
with gzip.open('./pw-data/201701scripts_sample.json.gz', 'rb') as f:
    scripts = json.load(f)

with gzip.open('./pw-data/practices.json.gz', 'rb') as f:
    practices = json.load(f)

This data set comes from Britain's National Health Service. The `scripts` variable is a list of prescriptions issued by NHS doctors. Each prescription is represented by a dictionary with various data fields: `'practice'`, `'bnf_code'`, `'bnf_name'`, `'quantity'`, `'items'`, `'nic'`, and `'act_cost'`. 

In [6]:
scripts[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500}]

A [glossary of terms](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10686/Download-glossary-of-terms-for-GP-prescribing---presentation-level/pdf/PLP_Presentation_Level_Glossary_April_2015.pdf/) and [FAQ](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10048/FAQs-Practice-Level-Prescribingpdf/pdf/PLP_FAQs_April_2015.pdf/) is available from the NHS regarding the data. Below we supply a data dictionary briefly describing what these fields mean.

| Data field |Description|
|:----------:|-----------|
|`'practice'`|Code designating the medical practice issuing the prescription|
|`'bnf_code'`|British National Formulary drug code|
|`'bnf_name'`|British National Formulary drug name|
|`'quantity'`|Number of capsules/quantity of liquid/grams of powder prescribed|
| `'items'`  |Number of refills (e.g. if `'quantity'` is 30 capsules, 3 `'items'` means 3 bottles of 30 capsules)|
|  `'nic'`   |Net ingredient cost|
|`'act_cost'`|Total cost including containers, fees, and discounts|

The `practices` variable is a list of member medical practices of the NHS. Each practice is represented by a dictionary containing identifying information for the medical practice. Most of the data fields are self-explanatory. Notice the values in the `'code'` field of `practices` match the values in the `'practice'` field of `scripts`.

In [7]:
practices[:2]

[{'code': 'A81001',
  'name': 'THE DENSHAM SURGERY',
  'addr_1': 'THE HEALTH CENTRE',
  'addr_2': 'LAWSON STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 1HU'},
 {'code': 'A81002',
  'name': 'QUEENS PARK MEDICAL CENTRE',
  'addr_1': 'QUEENS PARK MEDICAL CTR',
  'addr_2': 'FARRER STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 2AW'}]

In [8]:
from collections import defaultdict
practices_by_code=defaultdict(list)
for practice in practices:
    practices_by_code[practice['code']].append(practice)

In [4]:
#practices_by_code.items()

In [5]:
#list(practices_by_code.items())[:5]

In the following questions we will ask you to explore this data set. You may need to combine pieces of the data set together in order to answer some questions. Not every element of the data set will be used in answering the questions.

## Question 1: summary_statistics

Our beneficiary data (`scripts`) contains quantitative data on the number of items dispensed (`'items'`), the total quantity of item dispensed (`'quantity'`), the net cost of the ingredients (`'nic'`), and the actual cost to the patient (`'act_cost'`). Whenever working with a new data set, it can be useful to calculate summary statistics to develop a feeling for the volume and character of the data. This makes it easier to spot trends and significant features during further stages of analysis.

Calculate the sum, mean, standard deviation, and quartile statistics for each of these quantities. Format your results for each quantity as a list: `[sum, mean, standard deviation, 1st quartile, median, 3rd quartile]`. We'll create a `tuple` with these lists for each quantity as a final result.

In [11]:
#len(scripts)
#total of item is 
total= sum([script['items'] for script in scripts])

In [12]:
NUM=len(scripts)
avg = total/NUM
avg

11.522744731217633

In [13]:
NUM=len(scripts)
(sum([(script['items'] - avg) ** 2 for script in scripts]) / NUM) **0.5


33.11216633980368

In [7]:
from math import sqrt
def describe(key):
    NUM=len(scripts)

    total = sum([script[key] for script in scripts])
    avg = total/NUM
    
    s = sqrt(sum([(script[key] - avg) ** 2 for script in scripts]) / NUM)
    
    vals = sorted([script[key] for script in scripts])
     
    q25 = vals[NUM // 4]
    med = vals[NUM // 2]
    q75 = vals[-NUM // 4]

    return (total, avg, s, q25, med, q75)
    

In [7]:
summary = [('items', describe('items')),
           ('quantity', describe('quantity')),
           ('nic', describe('nic')),
           ('act_cost', describe('act_cost'))]

In [8]:
summary

[('items', (4410054, 11.522744731217633, 33.11216633980368, 1, 3, 8)),
 ('quantity',
  (316356836, 826.5883059943667, 3872.1810146096263, 30, 120, 466)),
 ('nic',
  (29048309.790000338,
   75.89844899484315,
   197.5728266277507,
   7.7,
   22.62,
   65.94)),
 ('act_cost',
  (27053937.599999707,
   70.68748295124895,
   183.26731895303854,
   7.25,
   21.24,
   61.53))]

In [9]:
grader.score.pw__summary_statistics(summary)

Your score:  1.0


## Question 2: most_common_item

Often we are not interested only in how the data is distributed in our entire data set, but within particular groups -- for example, how many items of each drug (i.e. `'bnf_name'`) were prescribed? Calculate the total items prescribed for each `'bnf_name'`. What is the most commonly prescribed `'bnf_name'` in our data?

To calculate this, we first need to split our data set into groups corresponding with the different values of `'bnf_name'`. Then we can sum the number of items dispensed within in each group. Finally we can find the largest sum.

We'll use `'bnf_name'` to construct our groups. You should have *5619* unique values for `'bnf_name'`.

In [9]:
bnf_names = {script['bnf_name'] for script in scripts}
assert(len(bnf_names) == 5619)

We want to construct "groups" identified by `'bnf_name'`, where each group is a collection of prescriptions (i.e. dictionaries from `scripts`). We'll construct a dictionary called `groups`, using `bnf_names` as the keys. We'll represent a group with a `list`, since we can easily append new members to the group. To split our `scripts` into groups by `'bnf_name'`, we should iterate over `scripts`, appending prescription dictionaries to each group as we encounter them.

In [10]:
groups = {name: [] for name in bnf_names}
for script in scripts:
    # INSERT ...
    bnf_name=script['bnf_name']
    groups[bnf_name].append(script)

Now that we've constructed our groups we should sum up `'items'` in each group and find the `'bnf_name'` with the largest sum. The result, `max_item`, should have the form `[(bnf_name, item total)]`, e.g. `[('Foobar', 2000)]`.

In [12]:
max_item = [("", 0)]

In [13]:
bnf_name_items=[]
for bnf_name, data in groups.items():
    total=sum([x['items'] for x in data])
    bnf_name_items.append((bnf_name, total))

In [22]:
bnf_name_items[:10]

[('Mentholatum_Deep Heat Max Strgh Crm', 28),
 ('Doublebase_Emollient Shower Gel', 410),
 ('Ondansetron HCl_Tab 8mg', 32),
 ('Zetuvit E 20cm x 20cm Pfa Cellulose Dres', 44),
 ('Aspirin Disper_Tab 75mg', 78830),
 ('Mebeverine HCl_Cap 200mg M/R', 1747),
 ('Betameth Val/Neomycin Sulf_Oint0.1/0.5%', 10),
 ('Estradiol/Norethisterone_Patch 50/170mcg', 5),
 ('Doublebase_Emollient Wash Gel', 69),
 ('Xupad Dress Pad Ster 10cm x 20cm', 5)]

In [23]:
sorted(bnf_name_items)

[('3m Health Care_Cavilon Durable Barrier C', 1998),
 ('3m Health Care_Cavilon No Sting 1ml Barr', 247),
 ('3m Health Care_Cavilon No Sting 3ml Barr', 39),
 ('3m Health Care_Cavilon No Sting Barrier', 350),
 ('A.S Saliva Orthana Spy 50ml (App)', 25),
 ('A2A Spacer', 46),
 ('A2A Spacer + Sml/Med Mask', 36),
 ('AMI_Suportx Ostomy & A/Support Ladies Br', 2),
 ('AMI_Suportx Ostomy/Hernia Easy Peel Supp', 9),
 ('APO-go PFS_Inf 5mg/ml 10ml Pfs', 2),
 ('Abasaglar KwikPen_100u/ml 3ml Pf Pen', 97),
 ('Abasaglar_100u/ml 3ml Cart', 5),
 ('Abidec_Dps', 1233),
 ('Able Spacer', 276),
 ('Able Spacer + Sml/Med Mask', 77),
 ('Abtard_Tab 10mg M/R', 2),
 ('Abtard_Tab 20mg M/R', 8),
 ('Acamprosate Calc_Tab E/C 333mg', 388),
 ('Acarbose_Tab 100mg', 6),
 ('Acarbose_Tab 50mg', 51),
 ('Accrete D3_Tab', 5993),
 ('Aceclofenac_Tab 100mg', 5),
 ('Acemetacin_Cap 60mg', 5),
 ('Acenocoumarol_Tab 1mg', 39),
 ('Acetazolamide_Cap 250mg M/R', 44),
 ('Acetazolamide_Tab 250mg', 182),
 ('Acetic Acid_Ear Spy 2% 5ml', 256),


In [24]:
bnf_name_items_sorted=sorted(bnf_name_items, key=lambda tup: tup[1], reverse=True)

In [25]:
bnf_name_items_sorted[:5]

[('Omeprazole_Cap E/C 20mg', 113826),
 ('Paracet_Tab 500mg', 81762),
 ('Aspirin Disper_Tab 75mg', 78830),
 ('Simvastatin_Tab 40mg', 69612),
 ('Salbutamol_Inha 100mcg (200 D) CFF', 68160)]

In [1]:
max_item=[('Omeprazole_Cap E/C 20mg', 113826)]

**TIP:** If you are getting an error from the grader below, please make sure your answer conforms to the correct format of `[(bnf_name, item total)]`.

In [27]:
#max_item= bnf_name_items_sorted[:5]

In [15]:
grader.score.pw__most_common_item(max_item)

Your score:  1.0


**Challenge:** Write a function that constructs groups as we did above. The function should accept a list of dictionaries (e.g. `scripts` or `practices`) and a tuple of fields to `groupby` (e.g. `('bnf_name')` or `('bnf_name', 'post_code')`) and returns a dictionary of groups. The following questions will require you to aggregate data in groups, so this could be a useful function for the rest of the miniproject.

In [2]:
def group_by_field(data, fields):
    groups = {}
    for field in fields:
        for lst in data:
            for key in lst:
                if field == key:
                    if lst[key] in groups:
                        groups[lst[key]] += lst['items']
                    else:
                        groups[lst[key]] = lst['items']              
    return groups

In [3]:
def group_by_field(data, field):
    groups={}
    
    for d in data:
        key =tuple(d[f] for f in field)
        if key in groups:
            groups[key].append(d)
        else:
            groups[key]=[d]
            
    return groups

In [12]:
def sum_values(iterable, key):
    return sum ([data[key]for data in iterable])

In [13]:
sum_values(scripts, 'items')

4410054

In [17]:
#scripts[:3]

In [34]:
groups['Sod Algin/Pot Bicarb_Susp S/F']

[{'bnf_code': '0101021B0AAALAL',
  'items': 12,
  'practice': 'N81013',
  'bnf_name': 'Sod Algin/Pot Bicarb_Susp S/F',
  'nic': 64.51,
  'act_cost': 59.95,
  'quantity': 6300},
 {'bnf_code': '0101021B0AAALAL',
  'items': 12,
  'practice': 'N81029',
  'bnf_name': 'Sod Algin/Pot Bicarb_Susp S/F',
  'nic': 74.24,
  'act_cost': 69.16,
  'quantity': 7250},
 {'bnf_code': '0101021B0AAALAL',
  'items': 21,
  'practice': 'N81062',
  'bnf_name': 'Sod Algin/Pot Bicarb_Susp S/F',
  'nic': 113.66,
  'act_cost': 105.83,
  'quantity': 11100},
 {'bnf_code': '0101021B0AAALAL',
  'items': 25,
  'practice': 'N81085',
  'bnf_name': 'Sod Algin/Pot Bicarb_Susp S/F',
  'nic': 148.47,
  'act_cost': 138.22,
  'quantity': 14500},
 {'bnf_code': '0101021B0AAALAL',
  'items': 24,
  'practice': 'N81088',
  'bnf_name': 'Sod Algin/Pot Bicarb_Susp S/F',
  'nic': 137.21,
  'act_cost': 127.7,
  'quantity': 13400},
 {'bnf_code': '0101021B0AAALAL',
  'items': 4,
  'practice': 'N81010',
  'bnf_name': 'Sod Algin/Pot Bicarb_

In [ ]:
def group_by_field(data, fields):
    groups = {}
    return groups

NameError: name 'scripts' is not defined

## Question 3: postal_totals

Our data set is broken up among different files. This is typical for tabular data to reduce redundancy. Each table typically contains data about a particular type of event, processes, or physical object. Data on prescriptions and medical practices are in separate files in our case. If we want to find the total items prescribed in each postal code, we will have to _join_ our prescription data (`scripts`) to our clinic data (`practices`).

Find the total items prescribed in each postal code, representing the results as a list of tuples `(post code, total items prescribed)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes. Only include post codes if there is at least one prescription from a practice in that post code.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code.

We can join `scripts` and `practices` based on the fact that `'practice'` in `scripts` matches `'code'` in `practices`. However, we must first deal with the repeated values of `'code'` in `practices`. We want the alphabetically first postal codes.

In [ ]:
practices[:5]

In [ ]:
list(practices_by_code.items())[:5]

In [8]:
practice_postal = {}
for practice in practices:
    if practice['code'] in practice_postal:
        practice_postal[practice['code']] = ...
    else:
        practice_postal[practice['code']] = ...

NameError: name 'practices' is not defined

In [7]:
practice_postal = {}
for practice in practices:
    if practice['code'] in practice_postal:
        practice_postal[practice['code']] = min(practice['post_code'],practice_postal[practice['code']])
    else:
        practice_postal[practice['code']] = practice['post_code']

NameError: name 'practices' is not defined

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [14]:
assert practice_postal['K82019'] == 'HP21 8TR'

Now we can join `practice_postal` to `scripts`.

In [ ]:
scripts[:]

In [ ]:
scripts[4],["quantity"]

In [ ]:
scripts[4]

In [ ]:
scripts[4]['quantity']  #slice the quabtity in position 4 of list of dictionaries

In [ ]:
joined = scripts[:]
for script in joined:
    script['post_code'] = ...

In [15]:
joined = scripts[:]
for script in joined:
    script['post_code'] = practice_postal[script['practice']]

In [16]:
joined[:4]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000,
  'post_code': 'SK11 6JL'},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500,
  'post_code': 'SK11 6JL'},
 {'bnf_code': '0101021B0AAALAL',
  'items': 12,
  'practice': 'N81013',
  'bnf_name': 'Sod Algin/Pot Bicarb_Susp S/F',
  'nic': 64.51,
  'act_cost': 59.95,
  'quantity': 6300,
  'post_code': 'SK11 6JL'},
 {'bnf_code': '0101021B0AAAPAP',
  'items': 3,
  'practice': 'N81013',
  'bnf_name': 'Sod Alginate/Pot Bicarb_Tab Chble 500mg',
  'nic': 9.21,
  'act_cost': 8.55,
  'quantity': 180,
  'post_code': 'SK11 6JL'}]

Finally we'll group the prescription dictionaries in `joined` by `'post_code'` and sum up the items prescribed in each group, as we did in the previous question.

In [ ]:
items_by_post = ...

In [17]:
items_by_post = group_by_field(joined, ('post_code',))

In [ ]:
items_by_post.keys()

In [18]:
`

[{'bnf_code': '0101010R0BCAAAB',
  'items': 2,
  'practice': 'G81013',
  'bnf_name': 'Infacol_Susp 40mg/ml S/F',
  'nic': 5.42,
  'act_cost': 5.04,
  'quantity': 100,
  'post_code': 'TN34 1BA'},
 {'bnf_code': '0101021B0AAALAL',
  'items': 15,
  'practice': 'G81013',
  'bnf_name': 'Sod Algin/Pot Bicarb_Susp S/F',
  'nic': 79.87,
  'act_cost': 74.22,
  'quantity': 7800,
  'post_code': 'TN34 1BA'},
 {'bnf_code': '0101021B0AAAPAP',
  'items': 4,
  'practice': 'G81013',
  'bnf_name': 'Sod Alginate/Pot Bicarb_Tab Chble 500mg',
  'nic': 14.33,
  'act_cost': 13.52,
  'quantity': 280,
  'post_code': 'TN34 1BA'},
 {'bnf_code': '0101021B0BEADAJ',
  'items': 17,
  'practice': 'G81013',
  'bnf_name': 'Gaviscon Infant_Sach 2g (Dual Pack) S/F',
  'nic': 266.06,
  'act_cost': 246.62,
  'quantity': 828,
  'post_code': 'TN34 1BA'},
 {'bnf_code': '0101021B0BEAIAL',
  'items': 17,
  'practice': 'G81013',
  'bnf_name': 'Gaviscon Advance_Liq (Aniseed) (Reckitt)',
  'nic': 94.9,
  'act_cost': 88.07,
  'quant

In [28]:
post = [(k, v) for k, v in sorted(items_by_post.items())]
postal_totals = post[:100]

In [1]:
#print(postal_totals)
#print(post[:100])

In [33]:
post=[(post_code[0], sum_values(data, 'items')) for post_code, data in sorted(items_by_post.items())]

In [34]:
postal_totals=post[:100]

In [2]:
postal_totals=[(post_code[0], sum_values(data, 'items')) for post_code, data in items_by_post.items()]

NameError: name 'items_by_post' is not defined

In [35]:
#postal_totals = [('B11 4BW', 20673)] * 100

grader.score.pw__postal_totals(postal_totals)

Your score:  1.0


## Question 4: items_by_region

Now we'll combine the techniques we've developed to answer a more complex question. Find the most commonly dispensed item in each postal code, representing the results as a list of tuples (`post_code`, `bnf_name`, amount dispensed as proportion of total). Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** We'll continue to use the `joined` variable we created before, where we've chosen the alphabetically first postal code for each practice. Additionally, some postal codes will have multiple `'bnf_name'` with the same number of items prescribed for the maximum. In this case, we'll take the alphabetically first `'bnf_name'`.

Now we need to calculate the total items of each `'bnf_name'` prescribed in each `'post_code'`. Use the techniques we developed in the previous questions to calculate these totals. You should have 141196 `('post_code', 'bnf_name')` groups.

In [38]:
grouped_by_bnf_name_and_post_code = group_by_field(joined, ('post_code', 'bnf_name'))

In [39]:
grouped_by_bnf_name_and_post_code[('SK11 6JL', 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F')]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000,
  'post_code': 'SK11 6JL'},
 {'bnf_code': '0101010G0AAABAB',
  'items': 1,
  'practice': 'N81029',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 2.99,
  'act_cost': 2.78,
  'quantity': 500,
  'post_code': 'SK11 6JL'},
 {'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81088',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000,
  'post_code': 'SK11 6JL'}]

In [40]:
total_items_by_bnf_post =[]
for k, v in grouped_by_bnf_name_and_post_code.items():
    total_items_by_bnf_post.append((k, sum_values(v, 'items')))


In [41]:

[(k, sum_values(v, 'items')) for k, v in grouped_by_bnf_name_and_post_code.items()]


[(('SK11 6JL', 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F'), 5),
 (('SK11 6JL', 'Alginate_Raft-Forming Oral Susp S/F'), 3),
 (('SK11 6JL', 'Sod Algin/Pot Bicarb_Susp S/F'), 94),
 (('SK11 6JL', 'Sod Alginate/Pot Bicarb_Tab Chble 500mg'), 9),
 (('SK11 6JL', 'Gaviscon Infant_Sach 2g (Dual Pack) S/F'), 41),
 (('SK11 6JL', 'Gaviscon Advance_Liq (Aniseed) (Reckitt)'), 98),
 (('SK11 6JL', 'Gaviscon Advance_Tab Chble Mint(Reckitt)'), 16),
 (('SK11 6JL', 'Gaviscon Advance_Liq (Peppermint) S/F'), 65),
 (('SK11 6JL', 'Peptac_Liq (Peppermint) S/F'), 14),
 (('SK11 6JL', 'Alverine Cit_Cap 60mg'), 10),
 (('SK11 6JL', 'Hyoscine Butylbrom_Inj 20mg/ml 1ml Amp'), 3),
 (('SK11 6JL', 'Hyoscine Butylbrom_Tab 10mg'), 153),
 (('SK11 6JL', 'Mebeverine HCl_Tab 135mg'), 84),
 (('SK11 6JL', 'Mebeverine HCl_Cap 200mg M/R'), 194),
 (('SK11 6JL', 'Peppermint Oil_Cap E/C 0.2ml'), 42),
 (('SK11 6JL', 'Peppermint Oil_Cap E/C 0.2ml M/R'), 5),
 (('SK11 6JL', 'Colpermin_Cap E/C 0.2ml M/R'), 6),
 (('SK11 6JL', 'Ispag/Mebeverin

In [42]:
total_items_by_bnf_post[:4]

[(('SK11 6JL', 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F'), 5),
 (('SK11 6JL', 'Alginate_Raft-Forming Oral Susp S/F'), 3),
 (('SK11 6JL', 'Sod Algin/Pot Bicarb_Susp S/F'), 94),
 (('SK11 6JL', 'Sod Alginate/Pot Bicarb_Tab Chble 500mg'), 9)]

In [43]:
#total_items_by_bnf_post = ...
assert len(total_items_by_bnf_post) == 141196

Let's use `total_items` to find the maximum item total for each postal code. To do this, we will want to regroup `total_items_by_bnf_post` by `'post_code'` only, not by `('post_code', 'bnf_name')`. First let's turn `total_items` into a list of dictionaries (similar to `scripts` or `practices`) and then group it by `'post_code'`. You should have 118 groups in the resulting `total_items_by_post` after grouping `total_items` by `'post_code'`.

In [44]:
total_items_by_bnf`_post [0]

(('SK11 6JL', 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F'), 5)

In [45]:
total_items_by_bnf=[{'post_code': k[0], 'bnf_name':k[1], 'total': v}
                   for k, v in total_items_by_bnf_post ]

In [46]:
total_items_by_bnf[0]

{'post_code': 'SK11 6JL',
 'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
 'total': 5}

In [47]:
grouped_by_post_code=group_by_field(total_items_by_bnf, ('post_code',))
total_items_by_post=[(k[0], sum_values(v, 'total'))
                    for k, v in grouped_by_post_code.items()]

In [48]:
total_items = ...
assert len(total_items_by_post) == 118

In [49]:
total_items_by_post[:2]

[('SK11 6JL', 110071), ('CW5 5NX', 38797)]

Now we will aggregate the groups in `total_by_item_post` to create `max_item_by_post`. Some `'bnf_name'` have the same item total within a given postal code. Therefore, if more than one `'bnf_name'` has the maximum item total in a given postal code, we'll take the alphabetically first `'bnf_name'`. We can do this by [sorting](https://docs.python.org/2.7/howto/sorting.html) each group according to the item total and `'bnf_name'`.

In [50]:
list(grouped_by_post_code.values())[:2]

[[{'post_code': 'SK11 6JL',
   'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
   'total': 5},
  {'post_code': 'SK11 6JL',
   'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
   'total': 3},
  {'post_code': 'SK11 6JL',
   'bnf_name': 'Sod Algin/Pot Bicarb_Susp S/F',
   'total': 94},
  {'post_code': 'SK11 6JL',
   'bnf_name': 'Sod Alginate/Pot Bicarb_Tab Chble 500mg',
   'total': 9},
  {'post_code': 'SK11 6JL',
   'bnf_name': 'Gaviscon Infant_Sach 2g (Dual Pack) S/F',
   'total': 41},
  {'post_code': 'SK11 6JL',
   'bnf_name': 'Gaviscon Advance_Liq (Aniseed) (Reckitt)',
   'total': 98},
  {'post_code': 'SK11 6JL',
   'bnf_name': 'Gaviscon Advance_Tab Chble Mint(Reckitt)',
   'total': 16},
  {'post_code': 'SK11 6JL',
   'bnf_name': 'Gaviscon Advance_Liq (Peppermint) S/F',
   'total': 65},
  {'post_code': 'SK11 6JL',
   'bnf_name': 'Peptac_Liq (Peppermint) S/F',
   'total': 14},
  {'post_code': 'SK11 6JL', 'bnf_name': 'Alverine Cit_Cap 60mg', 'total': 10},
  {'post_code': 'SK11 6JL

In [51]:
max_item_by_post =[sorted(v, key=lambda x: (-x['total'], x['bnf_name']))[0]
                   for v in grouped_by_post_code.values()]

In [52]:
max_item_by_post[:5]

[{'post_code': 'SK11 6JL',
  'bnf_name': 'Omeprazole_Cap E/C 20mg',
  'total': 3219},
 {'post_code': 'CW5 5NX',
  'bnf_name': 'Omeprazole_Cap E/C 20mg',
  'total': 1419},
 {'post_code': 'CW1 3AW',
  'bnf_name': 'Omeprazole_Cap E/C 20mg',
  'total': 2364},
 {'post_code': 'CW7 1AT',
  'bnf_name': 'Omeprazole_Cap E/C 20mg',
  'total': 1655},
 {'post_code': 'CH65 6TG',
  'bnf_name': 'Lansoprazole_Cap 30mg (E/C Gran)',
  'total': 688}]

In [54]:
total_items_by_post[:5]

[('SK11 6JL', 110071),
 ('CW5 5NX', 38797),
 ('CW1 3AW', 64104),
 ('CW7 1AT', 43164),
 ('CH65 6TG', 25090)]

In [58]:
total_items_by_post_dict=dict(total_items_by_post)

In [59]:
total_items_by_post_dict['SK11 6JL']

110071

In [ ]:
#post code, bnf_name, ratio=total of bnf_name/total for that post code

In [ ]:
max_item_by_post = ...

In [67]:
max_item_by_post_all=[(x['post_code'], x['bnf_name'], x['total']/ total_items_by_post_dict[x["post_code"]])
     for x in max_item_by_post]

In [68]:
max_items_by_post_all[:5]

[('SK11 6JL', 'Omeprazole_Cap E/C 20mg', 0.029244760200234393),
 ('CW5 5NX', 'Omeprazole_Cap E/C 20mg', 0.036574992911823076),
 ('CW1 3AW', 'Omeprazole_Cap E/C 20mg', 0.03687757394234369),
 ('CW7 1AT', 'Omeprazole_Cap E/C 20mg', 0.038342136965990176),
 ('CH65 6TG', 'Lansoprazole_Cap 30mg (E/C Gran)', 0.027421283379832604)]

In order to express the item totals as a proportion of the total amount of items prescribed across all `'bnf_name'` in a postal code, we'll need to use the total items prescribed that we previously calculated as `items_by_post`. Calculate the proportions for the most common `'bnf_names'` for each postal code. Format your answer as a list of tuples: `[(post_code, bnf_name, total)]`

In [73]:
#items_by_region = [('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.0341508247)] * 100
items_by_region=sorted(max_item_by_post_all)[:100]

In [74]:
grader.score.pw__items_by_region(items_by_region)

Your score:  1.0


*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*